In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from lxml import etree
import time

# get player name
def get_player_name():
    player_list = []
    player = sp.find_all(class_ = 'full-3fV3c9pF')
    for i in range(0, len(player), 2):
        player_list.append(player[i].text + ' ' + player[i+1].text)
    return player_list

# get player's position
def get_player_position():
    position_list = []
    position = sp.find_all(class_ = 'position-28TbwVOg')
    for i in position:
        position_list.append(i.text)
    return position_list

def get_column_name():
    click_standard_button()
    sp = soup(driver.page_source, 'lxml')
    
    data_column_name = ['PLAYER', 'POSITION', 'TEAM']
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
    temp = sp.find_all(class_ = 'bui-text cellheader selected-1vxxHvFg bui-text')
    for i in range(0, len(temp), 2):
        data_column_name.append(temp[i].text)

    click_expanded_button()
    sp = soup(driver.page_source, 'lxml')
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
        
    return data_column_name

# click standard button
def click_standard_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    #time.sleep(0.5)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    time.sleep(2)
    sp = soup(driver.page_source, 'lxml')

# click expanded button
def click_expanded_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    #time.sleep(0.5)    
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    time.sleep(2)
    sp = soup(driver.page_source, 'lxml')

# click next page button
def click_next_page_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[3]/div[2]/div/div/div[2]/button').click()
    #time.sleep(0.5)
    #driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[3]/div[2]/div/div/div[2]/button').click()
    time.sleep(2)
    sp = soup(driver.page_source, 'lxml')

# get standard data
def get_data():
    data = []
    
    click_standard_button()
    sp = soup(driver.page_source, 'lxml')
    standard_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        standard_data.append(i.text)
    
    click_expanded_button()
    sp = soup(driver.page_source, 'lxml')
    expanded_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        expanded_data.append(i.text)

    for i in range(len(standard_data)//17):
        data.extend(standard_data[i*17:i*17+17])
        data.extend(expanded_data[i*16+1:i*16+16])

    return data

In [2]:
urls = pd.read_csv('mlb_stats_urls.csv', encoding='utf-8')

for year in range(2016, 2017):
    for link in urls['link']:
        df_list = []
        for page in range(1, 4):
            url = link + str(year) + '?page=' + str(page) + '&playerPool=ALL'
            driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')
            html = driver.get(url)
            sp = soup(driver.page_source, 'lxml')
            column_name = get_column_name()
            try:
                #while True:
                sp = soup(driver.page_source, 'lxml')
                print('page start')
                time.sleep(1)
                name = get_player_name()
                time.sleep(1)
                position = get_player_position()
                time.sleep(1)
                current_data = get_data()
                time.sleep(1)
                for i in range(len(name)):
                    row = []
                    row.append(name[i])
                    row.append(position[i])
                    row.extend(current_data[i*32:i*32+32])
                    df_list.append(row)
                time.sleep(1)
                #click_next_page_button()
            except:
                print('This is the last page :)')
        df = pd.DataFrame(df_list, columns=column_name)
        df.to_csv(f'mlb_stats/{year}/{link[26:-1]}_{year}.csv', index=False, encoding='utf-8')
        print(df)

C:\Users\user\AppData\Local\Temp\ipykernel_27072\1062187841.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
               PLAYER POSITION TEAM    G   AB    R    H  2B 3B  HR  ... GO/AO  \
0      Jhoulys Chacin        P  LAA   29    2    1    1   0  0   0  ...  0.00   
1          Mike Trout       CF  LAA  159  549  123  173  32  5  29  ...  0.98   
2        Geovany Soto        C  LAA   26   78   11   21   5  0   4  ...  0.81   
3           C.J. Cron       1B  LAA  116  407   51  113  25  2  16  ...  0.91   
4         Jefry Marte       1B  LAA   88  258   38   65  14  0  15  ...  0.99   
5        Kole Calhoun       RF  LAA  157  594   91  161  35  5  18  ...  0.82   
6       Albert Pujols       DH  LAA  152  593   71  159  19  0  31  ...  0.99   
7       Yunel Escobar       3B  LAA  132  517   68  157  28  1   5  ...  1.91   
8       Juan Graterol        C  LAA    9   14    2    4   2  0   0  ...  3.00   
9   Andrelton Simmons       SS  LAA  124  448   48  126  22  2   4  ...  1.63   
10         Jett Bandy        C  LAA   70  209   23   49   9  0   8  ...  0.4

C:\Users\user\AppData\Local\Temp\ipykernel_27072\1062187841.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
               PLAYER POSITION TEAM    G   AB    R    H  2B 3B  HR  ... GO/AO  \
0         Jose Altuve       2B  HOU  161  640  108  216  42  5  24  ...  0.97   
1         Evan Gattis       DH  HOU  128  447   58  112  19  0  32  ...  0.97   
2     George Springer       RF  HOU  162  644  116  168  29  5  29  ...  1.43   
3       Luis Valbuena       3B  HOU   90  292   38   76  17  1  13  ...  0.84   
4       Carlos Correa       SS  HOU  153  577   76  158  36  3  20  ...  1.44   
5        Alex Bregman       3B  HOU   49  201   31   53  13  3   8  ...  0.54   
6   Teoscar Hernandez       LF  HOU   41  100   15   23   7  0   4  ...  1.00   
7     Marwin Gonzalez       1B  HOU  141  484   55  123  26  3  13  ...  1.29   
8        Jason Castro        C  HOU  113  329   41   69  16  3  11  ...  1.42   
9        Yuli Gurriel       3B  HOU   36  130   13   34   7  0   3  ...  0.77   
10    Lance McCullers        P  HOU   14    3    1    1   0  0   0  ...  1.0

C:\Users\user\AppData\Local\Temp\ipykernel_27072\1062187841.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


...

page start
page start
page start
               PLAYER POSITION TEAM    G   AB    R    H  2B 3B  HR  ... GO/AO  \
0        Jose Berrios        P  MIN   14    1    0    1   0  0   0  ...  -.--   
1        Tommy Milone        P  MIN   19    2    0    2   0  0   0  ...  -.--   
2        Brian Dozier       2B  MIN  155  615  104  165  35  5  42  ...  0.65   
3       Kennys Vargas       1B  MIN   47  152   27   35  11  0  10  ...  0.79   
4     Robbie Grossman       LF  MIN   99  332   49   93  19  1  11  ...  0.88   
5         Miguel Sano       3B  MIN  116  437   57  103  22  1  25  ...  0.67   
6       Eduardo Nunez       SS  MIN   91  371   49  110  15  1  12  ...  1.06   
7       Jorge Polanco       SS  MIN   69  245   24   69  15  4   4  ...  0.65   
8           Joe Mauer       1B  MIN  134  494   68  129  22  4  11  ...  1.77   
9          Max Kepler       RF  MIN  113  396   52   93  20  2  17  ...  1.18   
10     Trevor Plouffe       3B  MIN   84  319   35   83  13  1  12  ...  0.9

C:\Users\user\AppData\Local\Temp\ipykernel_27072\1062187841.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
             PLAYER POSITION TEAM    G   AB   R    H  2B 3B  HR  ... GO/AO  \
0       Matt Albers        P  CWS   58    1   1    1   1  0   0  ...  -.--   
1   Anthony Ranaudo        P  CWS    7    3   1    1   0  0   1  ...  1.00   
2     Matt Davidson       DH  CWS    1    2   1    1   0  0   0  ...  -.--   
3    Charlie Tilson       CF  CWS    1    2   0    1   0  0   0  ...  1.00   
4        Jose Abreu       1B  CWS  159  624  67  183  32  1  25  ...  1.04   
5     Melky Cabrera       LF  CWS  151  591  70  175  42  5  14  ...  0.95   
6        Adam Eaton       RF  CWS  157  619  91  176  29  9  14  ...  1.64   
7      Todd Frazier       3B  CWS  158  590  89  133  21  0  40  ...  0.69   
8      Tim Anderson       SS  CWS   99  410  57  116  22  6   9  ...  1.83   
9        Alex Avila        C  CWS   57  169  19   36   6  0   7  ...  2.73   
10   Justin Morneau       DH  CWS   58  203  16   53  14  1   6  ...  1.33   
11   Tyler Saladino       2B  C

C:\Users\user\AppData\Local\Temp\ipykernel_27072\1062187841.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
             PLAYER POSITION TEAM    G   AB   R    H  2B 3B  HR  ... GO/AO  \
0    Chris Parmelee       1B  NYY    6    8   4    4   1  0   2  ...  0.00   
1      Gary Sanchez        C  NYY   53  201  34   60  12  0  20  ...  1.69   
2      Billy Butler       DH  NYY   12   29   3   10   2  0   1  ...  1.00   
3    Carlos Beltran       RF  NYY   99  359  50  109  21  0  22  ...  0.86   
4      Tyler Austin       1B  NYY   31   83   7   20   3  0   5  ...  1.25   
5    Didi Gregorius       SS  NYY  153  562  68  155  32  2  20  ...  0.82   
6      Brian McCann        C  NYY  130  429  56  104  13  0  20  ...  0.73   
7    Starlin Castro       2B  NYY  151  577  63  156  29  1  21  ...  1.29   
8     Chase Headley       3B  NYY  140  467  58  118  18  1  14  ...  0.95   
9    Donovan Solano       2B  NYY    9   22   5    5   2  0   1  ...  1.33   
10    Brett Gardner       LF  NYY  148  547  80  143  22  6   7  ...  1.36   
11  Jacoby Ellsbury       CF  N

In [ ]:
driver.quit()